# Importing Libraries

In [14]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd

# Model

In [4]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [6]:
tokenizer

BertTokenizerFast(name_or_path='nlptown/bert-base-multilingual-uncased-sentiment', vocab_size=105879, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

# Encode and calculate the sentiment

In [7]:
tokens = tokenizer.encode("Hi there, I'm feeling great!!", return_tensors='pt')

In [11]:
results = model(tokens)

In [12]:
results

SequenceClassifierOutput(loss=None, logits=tensor([[-2.3318, -2.2624, -0.5070,  1.6017,  2.7476]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [13]:
results.logits

tensor([[-2.3318, -2.2624, -0.5070,  1.6017,  2.7476]],
       grad_fn=<AddmmBackward0>)

In [18]:
int(torch.argmax(results.logits))+1

5

# Collecting Reviews

In [28]:
r = requests.get('https://www.trustpilot.com/review/swiggy.com')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*typography_heading-s__f7029.*')
results = soup.find_all('p', {'class': regex})
reviews = [result.text for result in results]

In [33]:
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [35]:
results

[<p class="comment__09f24__D0cxf css-qgunke"><span class="raw__09f24__T4Ezm" lang="en">Some of the best Milkshakes me and my daughter ever tasted. MMMMMM HMMMMMMMM.</span></p>,
 <p class="comment__09f24__D0cxf css-qgunke"><span class="raw__09f24__T4Ezm" lang="en">Six of us met here for breakfast before our walk to Manly. We were enjoying visiting with each other so much that I apologize for not taking any photos. <br/>We all enjoyed our food, as well as our coffee and tea drinks.<br/>We were greeted immediately by a friendly server asking if we would like to sit inside or out. We said we would like inside, but weren't exactly sure how many were joining us yet- at least 4. We were told this was no problem, the more the merrier. A few minutes later when 4 more joined our party and we explained to the server we had 6, he just quickly switched our table. <br/>I really enjoyed my serenity tea, just what I needed after a long flight in from Sfo that morning. Everyone else were more intereste

# Load Reviews into DataFrame and check the score

In [36]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [37]:
df.head()

,review
0,Some of the best Milkshakes me and my daughter...
1,Six of us met here for breakfast before our wa...
2,Great place with delicious food and friendly s...
3,Great food amazing coffee and tea. Short walk ...
4,It was ok. Had coffee with my friends. I'm new...


In [38]:
df['review'].iloc[0]

'Some of the best Milkshakes me and my daughter ever tasted. MMMMMM HMMMMMMMM.'

In [39]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [41]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [42]:
df['review']

0    Some of the best Milkshakes me and my daughter...
1    Six of us met here for breakfast before our wa...
2    Great place with delicious food and friendly s...
3    Great food amazing coffee and tea. Short walk ...
4    It was ok. Had coffee with my friends. I'm new...
5    Ricotta hot cakes! These were so yummy. I ate ...
6    Great staff and food.  Must try is the pan fri...
7    We came for brunch twice in our week-long visi...
8    I came to Social brew cafe for brunch while ex...
9    It was ok. The coffee wasn't the best but it w...
Name: review, dtype: object